In [23]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append(os.path.join(os.getcwd(), ".."))

from models.sp500.fundamentals import FundamentalStockAnalyzer
import logging
from datetime import datetime, timedelta
import json

logging.basicConfig(level=logging.CRITICAL)  # or DEBUG for more detail
fas = await FundamentalStockAnalyzer.create()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
print("=== Processing data from S&P 500 ===")
recommended_stocks = await fas.get_recommended_stocks(max_concurrent=5)

print("\n=== Top 10 Recommended Stocks (Best Scores) ===")
for i, stock in enumerate(recommended_stocks[:10], 1):
    ticker = stock["ticker"]
    name = stock.get("shortName", "N/A")
    total_score = stock["total_score"]
    annual_change_percent = stock.get("annualChangePercent", "N/A")
    six_month_change_percent = stock.get("sixMonthChangePercent", "N/A")
    three_month_change_percent = stock.get("threeMonthChangePercent", "N/A")
    one_month_change_percent = stock.get("oneMonthChangePercent", "N/A")
    one_week_change_percent = stock.get("oneWeekChangePercent", "N/A")

    # Show some key financial metrics
    operating_cf = stock.get("operatingCashflow", "N/A")
    profit_margins = stock.get("profitMargins", "N/A")
    roa = stock.get("returnOnAssets", "N/A")
    enterprise_value = stock.get("enterpriseValue", "N/A")

    print(f"{i:2d}. {ticker} - {name}")
    print(f"     Total Score: {total_score}")
    print(
        f"     Annual Change: {annual_change_percent}% | 6 Month Change: {six_month_change_percent}% | 3 Month Change: {three_month_change_percent}% | 1 Month Change: {one_month_change_percent}% | 1 Week Change: {one_week_change_percent}%"
    )
    print(
        f"     Operating CF: ${operating_cf:,} | Profit Margin: {profit_margins} | ROA: {roa}"
    )
    print(f"     Enterprise Values: ${enterprise_value:,}")
    print()

=== Processing data from S&P 500 ===

=== Top 10 Recommended Stocks (Best Scores) ===
 1. ACN - Accenture plc
     Total Score: 127
     Annual Change: -26.93% | 6 Month Change: -20.2% | 3 Month Change: -15.04% | 1 Month Change: -6.94% | 1 Week Change: 1.13%
     Operating CF: $10,949,683,200 | Profit Margin: 0.11607 | ROA: 0.112449996
     Enterprise Values: $147,020,398,592

 2. CMG - Chipotle Mexican Grill, Inc.
     Total Score: 113
     Annual Change: -30.68% | 6 Month Change: -18.26% | 3 Month Change: -24.84% | 1 Month Change: -9.1% | 1 Week Change: 2.69%
     Operating CF: $2,091,666,048 | Profit Margin: 0.13317999 | ROA: 0.13668
     Enterprise Values: $55,930,875,904

 3. DHR - Danaher Corporation
     Total Score: 104
     Annual Change: -28.86% | 6 Month Change: -8.43% | 3 Month Change: -1.6% | 1 Month Change: -8.9% | 1 Week Change: 1.52%
     Operating CF: $6,168,999,936 | Profit Margin: 0.14213 | ROA: 0.04006
     Enterprise Values: $151,992,352,768

 4. ODFL - Old Dominio

In [25]:
# Get the correct path - stay within the project directory
project_root = os.getcwd()  # This should be /Users/gaz/zignolz-analytics-api
data_dir = os.path.join(project_root, "models", "data", "sp500")
print(f"Project root: {project_root}")
print(f"Target directory: {data_dir}")

# Ensure the data directory exists
os.makedirs(data_dir, exist_ok=True)

# Define the output file path
output_file = os.path.join(data_dir, "stock_recommendations_current_week.json")


# Function to check if a date is in the current week (midnight Friday to midnight Friday)
def is_current_week(date_str):
    try:
        file_date = datetime.fromisoformat(date_str.replace("Z", "+00:00"))
        if file_date.tzinfo:
            file_date = file_date.replace(tzinfo=None)

        now = datetime.now()

        # Find the most recent Friday (start of current week)
        if now.weekday() >= 4:  # Friday (4), Saturday (5), or Sunday (6)
            days_back_to_friday = now.weekday() - 4
        else:  # Monday (0) through Thursday (3)
            days_back_to_friday = now.weekday() + 3  # Go back to previous Friday

        most_recent_friday = now - timedelta(days=days_back_to_friday)
        current_week_start = most_recent_friday.replace(
            hour=0, minute=0, second=0, microsecond=0
        )
        current_week_end = current_week_start + timedelta(days=7)

        print(f"Today: {now.strftime('%A, %Y-%m-%d')}")
        print(f"Current week range: {current_week_start} to {current_week_end}")
        print(f"File date: {file_date}")

        return current_week_start <= file_date < current_week_end
    except Exception as e:
        print(f"Error parsing date: {e}")
        return False


# Check if file exists and handle accordingly
if os.path.exists(output_file):
    print(f"File exists: {output_file}")

    # Read existing file to check date
    try:
        with open(output_file, "r") as f:
            existing_data = json.load(f)

        existing_date = existing_data.get("generatedAt")
        if existing_date and is_current_week(existing_date):
            print("✅ File is from current week. No update needed.")
            print(
                f"📊 Existing file contains {existing_data.get('totalStocks', 0)} stocks"
            )
        else:
            print("📅 File is from a previous week. Creating backup and updating...")

            # Use the actual date from generatedAt for the backup filename
            if existing_date:
                existing_datetime = datetime.fromisoformat(
                    existing_date.replace("Z", "+00:00")
                )
                if existing_datetime.tzinfo:
                    existing_datetime = existing_datetime.replace(tzinfo=None)
                # Use the actual date from the file, not previous Monday
                backup_date_str = existing_datetime.strftime("%Y%m%d")
            else:
                # Fallback if no generatedAt date found
                backup_date_str = datetime.now().strftime("%Y%m%d")

            backup_filename = f"stock_recommendations_{backup_date_str}.json"
            backup_file = os.path.join(data_dir, backup_filename)

            print(f"📅 Original file date: {existing_date}")
            print(f"📦 Backup filename will be: {backup_filename}")

            # Rename existing file
            os.rename(output_file, backup_file)
            print(f"📦 Existing file renamed to: {backup_filename}")

            # Continue to save new data
    except Exception as e:
        print(f"⚠️ Error reading existing file: {e}. Will overwrite.")
else:
    print("📄 File doesn't exist. Creating new file.")

# Only proceed with saving if we need to update (file doesn't exist or is from previous week)
if not os.path.exists(output_file):
    # Prepare data for JSON export (excluding individual field scores)
    export_data = []
    for stock in recommended_stocks:
        stock_data = {
            "ticker": stock["ticker"],
            "companyName": stock.get("shortName", "N/A"),
            "totalScore": stock["total_score"],
            "fundamentals": {
                "annualChangePercent": stock.get("annualChangePercent"),
                "sixMonthChangePercent": stock.get("sixMonthChangePercent"),
                "threeMonthChangePercent": stock.get("threeMonthChangePercent"),
                "oneMonthChangePercent": stock.get("oneMonthChangePercent"),
                "oneWeekChangePercent": stock.get("oneWeekChangePercent"),
                "operatingCashflow": stock.get("operatingCashflow"),
                "profitMargins": stock.get("profitMargins"),
                "returnOnAssets": stock.get("returnOnAssets"),
                "enterpriseValue": stock.get("enterpriseValue"),
                "revenueGrowth": stock.get("revenueGrowth"),
                "operatingMargins": stock.get("operatingMargins"),
                "returnOnEquity": stock.get("returnOnEquity"),
            },
        }
        export_data.append(stock_data)

    # Create the output data structure
    output_data = {
        "generatedAt": datetime.now().isoformat(),
        "totalStocks": len(export_data),
        "recommendations": export_data,
    }

    try:
        with open(output_file, "w") as f:
            json.dump(output_data, f, indent=2, default=str)
        print(f"✅ Stock recommendations saved to: {output_file}")
        print(f"📊 Total stocks exported: {len(export_data)}")

        # Verify file was created in the correct location
        if os.path.exists(output_file):
            file_size = os.path.getsize(output_file)
            print(f"✅ File confirmed to exist with size: {file_size} bytes")
        else:
            print("❌ File was not created successfully")

    except Exception as e:
        print(f"❌ Error saving file: {e}")

Project root: /Users/gaz/zignolz-analytics-api
Target directory: /Users/gaz/zignolz-analytics-api/models/data/sp500
📄 File doesn't exist. Creating new file.
✅ Stock recommendations saved to: /Users/gaz/zignolz-analytics-api/models/data/sp500/stock_recommendations_current_week.json
📊 Total stocks exported: 10
✅ File confirmed to exist with size: 6066 bytes
